In [3]:
import pandas as pd
import numpy as np

In [4]:
bits_info = pd.DataFrame({
    "DRILL_BIT_NAME":["Buzz Drilldrin", "AstroBit", "Apollo", "ChallengDriller"],
    "CPR": [5000,3000,1000,10000],
    "CPF": [1.5,1,4,0],
    "CPH":[0,1500,2500,0]
})
bits_info

,DRILL_BIT_NAME,CPR,CPF,CPH
0,Buzz Drilldrin,5000,1.5,0
1,AstroBit,3000,1.0,1500
2,Apollo,1000,4.0,2500
3,ChallengDriller,10000,0.0,0


In [10]:
for i in range(1,21):
    csv = pd.read_csv(f'./Cleaned/Asteroid {i} - clean.csv')
    numeric_cols = csv.loc[:, csv.columns != 'DRILL_BIT_NAME']
    numeric_cols.loc[:][numeric_cols < 0] = np.NAN                         
    numeric_cols["DRILL_BIT_NAME"] = csv["DRILL_BIT_NAME"]
    data = numeric_cols.dropna()

    aggregated = (
        data
    )

    aggregated = aggregated.merge(bits_info)

    aggregated["NEXT_DEPTH"] = aggregated["BIT_DEPTH"].shift(-1)
    aggregated["NEXT_RATE"] = aggregated["RATE_OF_PENETRATION"].shift(-1)
    aggregated["PREV_ID"] = aggregated["DRILL_BIT_ID"].shift(1)
    aggregated["DEPTH_INTERVAL"] = aggregated["NEXT_DEPTH"] - aggregated["BIT_DEPTH"]


    def interval(x):
        #print(x["DEPTH_INTERVAL"])
        #print(x["BIT_DEPTH"])

        if x["BIT_DEPTH"] == x["NEXT_DEPTH"]:
            return 0

        if x["PREV_ID"] == x["DRILL_BIT_ID"] and (x["RATE_OF_PENETRATION"] / x["DEPTH_INTERVAL"]) != 0:
            return pow(x["RATE_OF_PENETRATION"] / x["DEPTH_INTERVAL"], -1) 
        elif (x["RATE_OF_PENETRATION"] / x["DEPTH_INTERVAL"]) != 0:
            return pow(x["RATE_OF_PENETRATION"] / x["DEPTH_INTERVAL"], -1) + (x["BIT_DEPTH"]/6000)
        else: 
            return 0

    aggregated["TIME_INTERVAL"] = aggregated.apply(interval, axis=1)

    def cpr_add(x):
        if x["PREV_ID"] == x["DRILL_BIT_ID"]:
            return (x["DEPTH_INTERVAL"] * x['CPF']) + (x["TIME_INTERVAL"] * x['CPH'])
        else:
            return x['CPR'] + (x["DEPTH_INTERVAL"] * x['CPF']) + (x["TIME_INTERVAL"] * x['CPH'])

    aggregated["cost"] = aggregated.apply(cpr_add, axis=1)

    path = f'./incrementalCSV/Asteroid{i}.csv'
    print(path)
    aggregated[['BIT_DEPTH','cost']]

./incrementalCSV/Asteroid1.csv
./incrementalCSV/Asteroid2.csv
./incrementalCSV/Asteroid3.csv
./incrementalCSV/Asteroid4.csv
./incrementalCSV/Asteroid5.csv
./incrementalCSV/Asteroid6.csv
./incrementalCSV/Asteroid7.csv
./incrementalCSV/Asteroid8.csv
./incrementalCSV/Asteroid9.csv
./incrementalCSV/Asteroid10.csv
./incrementalCSV/Asteroid11.csv
./incrementalCSV/Asteroid12.csv
./incrementalCSV/Asteroid13.csv
./incrementalCSV/Asteroid14.csv
./incrementalCSV/Asteroid15.csv
./incrementalCSV/Asteroid16.csv
./incrementalCSV/Asteroid17.csv
./incrementalCSV/Asteroid18.csv
./incrementalCSV/Asteroid19.csv
./incrementalCSV/Asteroid20.csv


In [115]:
aggregated['cost'].sum()

504827.69649616774